### Generating a toy-case topology

In [1]:
import numpy as np 
import cvxpy as cp

from itertools import combinations
from tqdm import tqdm

In [2]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,3),
    (0,6),
    (1,2),
    (1,5),
    (2,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f = B.T @ B

# Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [3]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [4]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [5]:
Y = U @ np.diag(H) @ U.T @ X

Y += np.random.normal(0, 10e-2, size=Y.shape)

In [6]:
np.trace(X.T @ L_f @ X)

12186.655518829697

In [7]:
np.trace(Y.T @ L_f @ Y)

134.33099570958169

____________

In [8]:
def solver(Y, d, edges, N):

    obj = 0
    trace = 0

    Fs = {
        edge: {
            edge[0]:None,
            edge[1]:None
        }
        for edge in edges
    }

    # Loop over edges to define variables and construct objective
    for edge in edges:
        u = edge[0]
        v = edge[1]

        Y_u = Y[u*d:(u+1)*d, :]
        Y_v = Y[v*d:(v+1)*d, :]

        # Define optimization variables for each edge
        Fs[edge][u] = cp.Variable((d, d))
        Fs[edge][v] = cp.Variable((d, d))

        # Update objective and trace
        obj += cp.norm(Fs[edge][u] @ Y_u - Fs[edge][v] @ Y_v, 'fro')**2
        trace += cp.trace(Fs[edge][u]) + cp.trace(Fs[edge][v])

    # Define the trace constraint
    trace_constraint = (trace == N)

    # Define the problem
    problem = cp.Problem(cp.Minimize(obj), [trace_constraint])

    # Solve the problem
    problem.solve(solver=cp.MOSEK,
                  mosek_params = { 'MSK_IPAR_INTPNT_SOLVE_FORM': 'MSK_SOLVE_PRIMAL' },
                  verbose=True)

    # Return the solution
    return Fs

In [9]:
Fs = solver(Y, d, edges,70)
B_hat = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = Fs[edge][u].value
    B_v = Fs[edge][v].value

    B_hat[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f_hat = B_hat.T @ B_hat
print(f'Average reconstruction error for trace barrier with N = {N}: {np.linalg.norm(L_f - L_f_hat) / L_f.size}')

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Aug 24 11:27:34 AM: Your problem has 144 variables, 1 constraints, and 0 parameters.
(CVXPY) Aug 24 11:27:34 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 24 11:27:34 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 24 11:27:34 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 24 11:27:34 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 24 11:27:34 AM: Compiling problem (target solver=MOSEK).
(C

In [10]:
np.trace(Y.T @ L_f @ Y)

134.33099570958169

In [11]:
np.trace(Y.T @ L_f_hat @ Y) 

136.16505557834063

_______________

In [12]:
def premultiplier(Xu, Xv):
    uu = np.linalg.inv(Xu @ Xu.T)
    uv = Xu @ Xv.T
    vv = np.linalg.inv(Xv @ Xv.T)
    vu = Xv @ Xu.T

    return (uu, uv, vv, vu)

def chi_u(uu, uv, vv, vu):

    return (uu @ uv + np.eye(uu.shape[0])) @ vv @ (vu @ uu @ uv @ vv - np.eye(uu.shape[0])) @ vu @ uu - uu

def chi_v(uu, uv, vv, vu):

    return (uu @ uv + np.eye(uu.shape[0])) @ vv @ (vu @ uu @ uv @ vv - np.eye(uu.shape[0]))

In [13]:
T = 0

maps_ = {
    edge : {
        edge[0] : np.zeros((d,d)),
        edge[1] : np.zeros((d,d))
    }
for edge in edges
}

In [14]:
for e in edges:
    
    u = e[0]
    v = e[1]

    X_u = Y[u*d:(u+1)*d,:]
    X_v = Y[v*d:(v+1)*d,:]
    
    uu, uv, vv, vu = premultiplier(X_u, X_v)

    maps_[e][u] = chi_u(uu, uv, vv, vu)
    maps_[e][v] = chi_v(uu, uv, vv, vu)
    
    T += np.trace(maps_[e][u]) + np.trace(maps_[e][v])

In [15]:
maps_

{(0,
  1): {0: array([[-1.14438111,  0.39792821, -0.22560556],
         [ 0.49617598, -0.78473805, -0.38389537],
         [-0.13320602, -0.4317819 , -0.8154591 ]]), 1: array([[-1.21055325,  0.38919862, -0.14265232],
         [ 0.29095085, -0.7541883 ,  0.03486775],
         [-0.23505186,  0.08275427, -0.71240735]])},
 (0,
  3): {0: array([[-0.88984643,  0.2973406 , -0.13143303],
         [ 0.24099473, -0.84302112, -0.31595405],
         [-0.20642857, -0.31338833, -0.74159045]]), 3: array([[-0.00896437, -0.04558887, -0.00994589],
         [ 0.010757  , -0.16012805, -0.21231497],
         [ 0.06504965, -0.21488069, -0.93588108]])},
 (0,
  6): {0: array([[-0.90335588,  0.43036218, -0.07486933],
         [ 0.36702726, -0.80054534, -0.26988445],
         [ 0.01744623, -0.45382203, -0.6649501 ]]), 6: array([[-0.63515863, -0.19391963,  0.36748948],
         [-0.13058471, -0.34990602, -0.35259368],
         [ 0.27517392, -0.1686561 , -0.6392148 ]])},
 (1,
  2): {1: array([[-1.1444694 ,  0.0675

In [16]:
maps_ = {
    edge : {
        edge[0] : 70/T * maps_[edge][edge[0]],
        edge[1] : 70/T * maps_[edge][edge[1]]
    }
for edge in edges
}

In [17]:
B_hat_2 = np.zeros((d*E, d*V))

for i, edge in enumerate(edges):

    u = edge[0] 
    v = edge[1] 

    B_u = maps_[edge][u]
    B_v = maps_[edge][v]

    B_hat_2[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat_2[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f_hat_2 = B_hat_2.T @ B_hat_2

In [18]:
np.trace(Y.T @ L_f_hat_2 @ Y)

146.23851723637512